In [ ]:
'''
yaml_selector.py

Utility to load a YAML tree and retrieve immediate children of a specified parent node.
'''
import numpy as np
import yaml
import os
from typing import Any, List, Optional

# A YAMLTree can be any Python structure returned by yaml.safe_load
YAMLTree = Any


def load_yaml(path: str, base_dir: str = '.') -> YAMLTree:
    """Load a YAML file from a relative or absolute path."""
    full_path = path if os.path.isabs(path) else os.path.join(base_dir, path)
    with open(full_path, 'r') as f:
        return yaml.safe_load(f)


def get_children(
    tree: YAMLTree,
    parent_path: str,
    base_path: str = ''
) -> Optional[List[str]]:
    """
    Return the list of immediate children keys or indices for the node at `parent_path`.

    - `tree` is the YAML structure.
    - `parent_path` is the dot/bracket-delimited path of the parent node (e.g. 'resource.aws_route_table').
    - `base_path` is used internally for recursion and should be left as default when calling.

    Returns a list of child labels (dict keys or list indices), or None if `parent_path` not found.
    """
    # Check if the current node is the target parent
    if base_path == parent_path or (base_path == '' and parent_path in ('<root>', '', 'root')):
        if isinstance(tree, dict):
            return list(tree.keys())
        if isinstance(tree, list):
            return [f"[{i}]" for i in range(len(tree))]
        return []  # Scalar node has no children

    # Recurse into dict
    if isinstance(tree, dict):
        for key, subtree in tree.items():
            child_path = f"{base_path}.{key}" if base_path else key
            result = get_children(subtree, parent_path, child_path)
            if result is not None:
                [print(item) for item in result] if isinstance(result, list) else None

                return result

    # Recurse into list
    elif isinstance(tree, list):
        for idx, item in enumerate(tree):
            child_path = f"{base_path}[{idx}]" if base_path else f"[{idx}]"
            result = get_children(item, parent_path, child_path)
            if result is not None:
                [print(item) for item in result] if isinstance(result, list) else None
                return result


    return None  # No match found

# Example Usage:
# --------------
# tree = load_yaml('resource.yaml', base_dir='relative/path')
# children = get_children(tree, 'resource.aws_route_table')
# print(children)  # e.g. ['private_kube']


In [34]:

# Example Usage
# -------------
# tree = load_yaml('resource.yaml', base_dir='relative/path')\#
# rels = get_parent_child_relationships(tree)
# for parent, children in rels:
#     print(f"Parent: {parent}\n  Children: {children}\n")

tree = load_yaml('yaml_combined/resource.yaml')
children = get_children(tree, 'resource')



